In [5]:
import numpy as np
import os
from pynwb import NWBFile, NWBHDF5IO, image
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, ImagingPlane
from pynwb.file import Subject
from pynwb.device import Device
import math

import statistics
import pandas as pd
import h5py #Provides methods to supplement NWB
import datajoint as dj #
import csv 
from datetime import datetime
from dateutil import tz
from pynwb import NWBHDF5IO
from matplotlib import pylab as plt

def connect_to_dj():
    dj.config['database.host'] = 'spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com' 
    dj.config['database.user'] = 'MarikeReimer'
    dj.config['database.password'] = 'uqHKL3YLMCG0'
    dj.conn()

    #Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
    current_schema = dj.schema('MarikeReimer', locals())
    return current_schema


schema = connect_to_dj()
print(schema)
schema

#Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
schema = dj.schema('MarikeReimer', locals())
schema


#Define Mouse table
@schema
class Mouse(dj.Manual):
    definition = """
    subject_id: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---' 
    genotype: enum('B6', 'BalbC', 'Unknown', 'Thy1-YFP')
    sex: enum('M', 'F', 'Unknown')
    species: varchar(128)
    strain: varchar(128)
    """

@schema
class Session(dj.Manual):
    definition = """
    ->Mouse
    identifier: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---' 
    surgery: varchar(128)
    pharmacology: varchar(128)
    """


@schema
class Dendrite(dj.Manual):
    definition = """
    ->Session
    dendrite_id: int                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---'
    soma_center_point: longblob
    proximal_dendrite_length: float
    medial_dendrite_length: float
    distal_dendrite_length: float
    """

@schema
class Image_segmentation(dj.Imported):
    definition = """
    ->Dendrite
    segmentation_name: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---'
    length: float
    volume: float
    surface_area:float
    spine_type: enum('mushroom', 'thin', 'disconnected','U')
    center_of_mass: longblob
    """
    def make(self, key):
        subject_id = key['subject_id']
        identifier = key['identifier']

        nwbfile_to_read = 'C:/Users/meowm/OneDrive/TanLab/DataJointTesting/NWBfiles/' + str(subject_id) + str(identifier) + '.nwb' #TODO: Remove hard coding
        #nwbfile_to_read = 'C:/Users/meowm/Downloads/NWBfiles/' + str(subject_id) + str(identifier) + '.nwb'

        print(nwbfile_to_read)
        with NWBHDF5IO(nwbfile_to_read, 'r') as io:
            nwbfile = io.read()     
            for group in nwbfile.processing["SpineData"]["ImageSegmentation"].children[:]:
                #print(group.name)
                if group.name.startswith("Mushroom"):
                    spine_type = 'mushroom'
                elif group.name.startswith("Thin"):
                    spine_type = 'thin'
                elif group.name.startswith("Disconnected"):
                    spine_type = 'disconnected'
                else:
                    spine_type = 'U'

                length = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].length.data[:]
                length = length[0]
                volume = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].volume.data[:]
                volume = volume[0]
                surface_area = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].surface_area.data[:]
                surface_area = surface_area[0]
                center_of_mass = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].center_of_mass.data[:]
                center_of_mass = center_of_mass[0]
                
                key['segmentation_name'] = group.name 
                key['length'] = length
                key['volume'] = volume
                key['surface_area'] = surface_area
                key['spine_type'] = spine_type
                key['center_of_mass'] = center_of_mass
                self.insert1(key)

@schema
class Distance_to_soma(dj.Computed):
    definition = """
    ->Image_segmentation
    ---
    distance_to_soma: float"""
    def make(self, key):
        center_of_mass = (Image_segmentation() & key).fetch1('center_of_mass')
        soma_center_point = (Dendrite() & key).fetch1('soma_center_point')
        distance_to_soma = math.dist(center_of_mass,soma_center_point)

        key['distance_to_soma'] = distance_to_soma
        self.insert1(key)

#Instantiate tables
mouse = Mouse()
session = Session()
dendrite = Dendrite()
image_segmentation = Image_segmentation()
distance_to_soma = Distance_to_soma()

print(distance_to_soma)

Schema `MarikeReimer`

*subject_id    *identifier    *dendrite_id   *segmentation_ distance_to_so
+------------+ +------------+ +------------+ +------------+ +------------+

 (Total: 0)



In [7]:
mouse.drop()

[2023-08-04 20:43:34,171][INFO]: `MarikeReimer`.`mouse` (0 tuples)
[2023-08-04 20:43:34,305][INFO]: `MarikeReimer`.`session` (0 tuples)
[2023-08-04 20:43:34,433][INFO]: `MarikeReimer`.`dendrite` (0 tuples)
[2023-08-04 20:43:34,561][INFO]: `MarikeReimer`.`_image_segmentation` (0 tuples)
[2023-08-04 20:43:34,700][INFO]: `MarikeReimer`.`__distance_to_soma` (0 tuples)


In [9]:
distance_to_soma

subject_id Primary keys above the '---',identifier Primary keys above the '---',dendrite_id Primary keys above the '---',segmentation_name Primary keys above the '---',distance_to_soma
L731,Dendrite1a,1,StubbySpine_23_plane_segmentaton,25.7012
L731,Dendrite1a,1,StubbySpine_24.001_plane_segmentaton,24.2863
L731,Dendrite1a,1,StubbySpine_24_plane_segmentaton,24.557
L731,Dendrite1a,1,StubbySpine_25_plane_segmentaton,23.6749
L731,Dendrite1a,1,StubbySpine_26_plane_segmentaton,32.8297
L731,Dendrite1a,1,StubbySpine_27_plane_segmentaton,28.0641
L731,Dendrite1a,1,StubbySpine_28_plane_segmentaton,37.9463
L731,Dendrite1a,1,ThinSpine_2_plane_segmentaton,26.336
L731,Dendrite1a,1,ThinSpine_3_plane_segmentaton,25.204
L731,Dendrite1a,1,ThinSpine_4_plane_segmentaton,48.8484


In [10]:
mouse*session*dendrite*image_segmentation*distance_to_soma

subject_id Primary keys above the '---',identifier Primary keys above the '---',dendrite_id Primary keys above the '---',segmentation_name Primary keys above the '---',genotype,sex,species,strain,surgery,pharmacology,soma_center_point,proximal_dendrite_length,medial_dendrite_length,distal_dendrite_length,length,volume,surface_area,spine_type,center_of_mass,distance_to_soma
L731,Dendrite1a,1,StubbySpine_23_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Contusion,Vehicle,=BLOB=,12.1678,37.8072,0.0,0.15186,0.00575145,0.186664,U,=BLOB=,25.7012
L731,Dendrite1a,1,StubbySpine_24.001_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Contusion,Vehicle,=BLOB=,12.1678,37.8072,0.0,0.0819186,0.00287346,0.103965,U,=BLOB=,24.2863
L731,Dendrite1a,1,StubbySpine_24_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Contusion,Vehicle,=BLOB=,12.1678,37.8072,0.0,0.126631,0.00670139,0.183515,U,=BLOB=,24.557
L731,Dendrite1a,1,StubbySpine_25_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Contusion,Vehicle,=BLOB=,12.1678,37.8072,0.0,0.250529,0.0327198,0.214995,U,=BLOB=,23.6749
L731,Dendrite1a,1,StubbySpine_26_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Contusion,Vehicle,=BLOB=,12.1678,37.8072,0.0,0.217024,0.0274928,0.427145,U,=BLOB=,32.8466
L731,Dendrite1a,1,StubbySpine_27_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Contusion,Vehicle,=BLOB=,12.1678,37.8072,0.0,0.308781,0.012968,0.157589,U,=BLOB=,28.0823
L731,Dendrite1a,1,StubbySpine_28_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Contusion,Vehicle,=BLOB=,12.1678,37.8072,0.0,0.143149,0.0174834,0.338095,U,=BLOB=,37.9648
L731,Dendrite1a,1,ThinSpine_2_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Contusion,Vehicle,=BLOB=,12.1678,37.8072,0.0,0.640246,0.0661667,0.43352,thin,=BLOB=,26.336
L731,Dendrite1a,1,ThinSpine_3_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Contusion,Vehicle,=BLOB=,12.1678,37.8072,0.0,0.251697,0.00344256,0.151821,thin,=BLOB=,25.3389
L731,Dendrite1a,1,ThinSpine_4_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Contusion,Vehicle,=BLOB=,12.1678,37.8072,0.0,1.49144,0.0216048,0.923622,thin,=BLOB=,48.8457


In [12]:
###
# Function to check if a given subject_id and identifier pair exists. TODO add to ClassDefinitions so that the CSV reader only works on your current file


def check_subject_identifier_existence(subject_id, identifier):
    mouse_query = Mouse & {'subject_id': subject_id}
    session_query = Session & {'subject_id': subject_id, 'identifier': identifier}
    
    mouse_exists = bool(mouse_query)
    session_exists = bool(session_query)
    
    return mouse_exists, session_exists

# Test the function with a subject_id and identifier pair
subject_id_to_check = 'L731'
identifier_to_check = 'Dendrite1a'

mouse_exists, session_exists = check_subject_identifier_existence(subject_id_to_check, identifier_to_check)

print(f"Mouse exists: {mouse_exists}")
print(f"Session exists: {session_exists}")

Mouse exists: True
Session exists: True


In [ ]:
frame = pd.DataFrame(subject*session*dendrite*(image_segmentation & 'length > "1"' & 'length <= "3"')*distance_to_soma)
print(frame)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/V11LargeSpinesSpinedata.csv")


In [ ]:
frame = pd.DataFrame(subject*session*dendrite)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/V11ShortDendriteList.csv")

In [ ]:
frame = pd.DataFrame(subject)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/SubjList.csv")

In [ ]:
frame = pd.DataFrame(subject*session*dendrite*image_segmentation*distance_to_soma)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/OP-09.csv")

In [31]:
def connect_to_dj():
    dj.config['database.host'] = 'spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com' 
    dj.config['database.user'] = 'MarikeReimer'
    dj.config['database.password'] = 'uqHKL3YLMCG0'
    dj.conn()

    #Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
    current_schema = dj.schema('MarikeReimer', locals())
    return current_schema


schema = connect_to_dj()
print(schema)

Schema `MarikeReimer`



In [32]:
Mouse.drop()

[2023-08-04 13:18:18,049][INFO]: `MarikeReimer`.`mouse` (0 tuples)
[2023-08-04 13:18:18,174][INFO]: `MarikeReimer`.`session` (0 tuples)
[2023-08-04 13:18:18,302][INFO]: `MarikeReimer`.`dendrite` (0 tuples)
[2023-08-04 13:18:23,571][INFO]: Dropped table `MarikeReimer`.`dendrite`
[2023-08-04 13:18:23,857][INFO]: Dropped table `MarikeReimer`.`session`
[2023-08-04 13:18:24,141][INFO]: Dropped table `MarikeReimer`.`mouse`
[2023-08-04 13:18:24,352][INFO]: Tables dropped. Restart kernel.
